## 노트북 내용

이 노트북은 watsonx.ai를 사용한 RAG (Retrieval Augmented Generation)을 시연하기 위한 것입니다. 여기에는 data retrieval, 지식 기본 정보에서 유사도검색 및 모델이 생성한 결과를 확인하는 내용이 포함되어 있습니다.
이를 잘 이해하기 위해서는 Python에 대한 기본 지식이 필요하며 모든 코드는 Python 3.10 으로 작성되어 있습니다.

### RAG (Retrieval Augmented Generation) 
Retrieval Augmented Generation (RAG)는 자연어로 지식 기반 데이터베이스에 질문이나 사실적인 정보를 사용하기를 원하는 다양한 usecase에 활용가능한 패턴입니다.

이 예제는 RAG의 가장 단순한 형태로서 세 가지 단계로 구성되어 있습니다. 

- 텍스트 데이타를 passage로 나누고 embedding하여 지식 기반 데이터베이스 구축
- 지식 기반 데이터베이스로부터 사용자 질문과 가장 유사한 passage들 추출
- 추출된 passage들을 large language model에 입력하여 사용자 질의에 대한 최종 답변 생성.

## 내용

이 노트북은 다음과 같은 단계로 구성되어 있습니다ㅏ.

- [환경설정](#setup)
- [지식기반 데이터베이스 구축](#build_base)
- [watsonx의 foundation model 접근 설정](#models)
- [사용자 질문에 대한 답변 생성](#predict)

<a id="setup"></a>
## 환경 설정

이노트북에 있는 샘플 코드를 실행하기 전에 다음 작업을 완료해야 합니다.

- 필요한 python package는 conda environment 혹은 python virtual environment에 python 3.10.12 기반의 독립적인 환경을 만든 후 pip install -r requirements_ibmcloud.txt를 사용해서 설치.  
- IBM Cloud상의 watsonx SAS 서비스에 접근할 수 있는 권한 정보 획득 필요.


In [ ]:
!pip install "langchain==0.2.6"
!pip install "ibm-watsonx-ai==1.0.10"
!pip install "langchain_ibm==0.1.8"
!pip install "langchain_community==0.2.6"
!pip install "sentence-transformers==3.0.1"
!pip install "chromadb==0.5.3"
!pip install "pydantic==2.8.2"
!pip install "langchain-huggingface==0.0.3"
!pip install "python-dotenv==1.0.1"

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain.llms import WatsonxLLM

### Watson Machine Learning에 연결 정보 확인

IBM Public Cloud 상의 watsonx SAS watsonx에 접근하기 위한 인증 정보를 입력한다.</br>
여기에는 다음과 같은 내용이 포함된다. <br/>
- url : watsonx 서비스의 url
- apikey : IBM Cloud의 계정에서 생성한 api key
- project id : watsonx 서비스 내에 있는 project id

In [ ]:
load_dotenv()

In [ ]:
# IBM Cloud 상의 watsonx SAS url은 provision된 region별로 정해져 있음. https://cloud.ibm.com/apidocs/watsonx-ai 참조.
project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": 'https://us-south.ml.cloud.ibm.com'
}

<a id="build_base"></a>
## 기본 지식 정보(knowledge base) 구축

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.
최신 기술인 RAG는 사용자 질의에 대한 sematic similarity를 계산하기 위해 기본 지식 정보에 대한 밀집 벡터 표현 (dense vector represenations)를 생성한다.
이 기본 예제에서는 "영화 분노의 도로-퓨리오사 사가"의 줄거리를 가져와서 chunk로 나누고 embedding 한 후에 Chroma db에 저장하는 것으로 기본 지식 정보를 구축한다.

### Document data 로딩

샘플 코드에서 사용할 데이타 파일은 단순 텍스트 파일이며 git clone으로 생성된 local repository에서 이 노트북이 존재하는 위치의 아래의 data 폴더에 저장되어있다.

In [ ]:
filename = os.path.join(os.getcwd(), 'data', '퓨리오사-줄거리.txt')

In [ ]:
loader = TextLoader(filename, encoding='UTF8')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

In [ ]:
texts[0:3]


### embedding function 생성

여기서는 huggingface에서 제공하는 기본 embedding 함수를 사용한다.
다른 embedding 함수를 사용할 수도 있으나 이 경우 embedding 함수와 사용하고자 하는 vector db (여기서는 Chroma)의 embedding size가 일치해야 한다.
참고로 Chroma db의 embedding size는 768 이다.

In [ ]:
embeddings = HuggingFaceEmbeddings()

### Vector DB (Chroma db) 생성

chunk로 나누어진 데이터와 embeddng 모델을 사용하여 Chroma db를 생성한다. 이 함수가 실행될 때 각 chunk들이 embedding되고 vector화된 데이터가 원본 텍스트와 함께 db내에 저장된다.

In [ ]:
docsearch = Chroma.from_documents(texts, embeddings)

<a id="models"></a>
## watsonx의 Foundation model 접근 설정

IBM watsonx의 foundation model들은 <a href="https://python.langchain.com/v0.2/docs/integrations/providers/ibm/#watsonxllm" target="_blank" rel="noopener no referrer">langchain에 의해 지원되는 LLM들의 목록</a>에 속한다.
여기서는 한글이 잘 동작한다고 평가되는 <a href="https://huggingface.co/mncai/llama2-13b-dpo-v7">mncai/llama2-13b-dpo-v7</a>를 사용한다.

WatsonxLLM class는 watsonx foundation model과 langchain 간의 interface를 제공해주는 class이다.

In [ ]:
ModelTypes.LLAMA_2_70B_CHAT.value

In [ ]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

model_id =  'mistralai/mistral-large' # 'meta-llama/llama-3-1-70b-instruct' # ModelTypes.LLAMA_2_70B_CHAT.value #
watsonx_llama2_korean = WatsonxLLM(
    model_id=model_id,
    url=wml_credentials.get("url"),
    apikey=wml_credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

### 사용자 질의 검색 테스트

앞서 구축한 지식 기반 정보로부터 사용자의 질의와 가장 유사한 정보를 검색해서 출력해 본다.

In [ ]:
retriever = docsearch.as_retriever(search_kwargs={'k': 5})
resutls = retriever.get_relevant_documents("퓨리오사가 한 쪽 팔을 잃게되는 경위가 뭔가요?")
resutls

<a id="predict"></a>
## RAG를 사용하여 사용자 질문에 대한 답변을 생성한다.

langchain의 `RetrievalQA` (question answering chain) 를 사용하여 RAG 작업을 자동으로 수행한다.

Prompt Template을 생성한뒤 기반 지식에 대한 Prompt 없이 LLM에 질문한다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_input}"),
    ("user", "{user_input}"),
])

In [ ]:
chain = chat_prompt | watsonx_llama2_korean

In [ ]:
chain.invoke({"system_input":"당신은 정직한 에이전트입니다.", "user_input":"퓨리오사가 한 쪽 팔을 잃게되는 이유는 뭔가요?"})

In [ ]:
chain.invoke({"system_input":"당신은 정직한 에이전트입니다.","user_input":"등장인물 중 디멘투스는 어떤 역할인가요??"})

In [ ]:
chain.invoke({"system_input":"당신은 정직한 에이전트입니다.","user_input":"퓨리오사는 어머니는 어떻게 되나요?"})

VectorDB에서 사용자 질문과 관련된 chunk를 찾은 후 Prompt에 삽입하고 LLM을 호출한다.

In [ ]:
resutls = retriever.get_relevant_documents("퓨리오사가의 한 쪽 팔이 잘리게 된 이유는 뭔가요?")
system_input = "당신은 정직한 에이전트 입니다. 반드시 주어진 기반지식에 근거해서 최대한 짧은 문장으로 답해주세요. 모르면 모른다고 답하세요. \n\n 기반지식:"
for doc in resutls:
    system_input += '\n' + doc.page_content

system_input

In [ ]:
chain.invoke({"system_input": system_input, "user_input":"퓨리오사가 한 쪽 팔을 잃게되는 이유는 뭔가요?"})

In [ ]:
resutls = retriever.get_relevant_documents("등장인물 중 디멘투스는 어떤 인물인가요?")
system_input = "당신은 정직한 에이전트 입니다. 반드시 주어진 기반지식에 근거해서 최대한 짧은 문장으로 답해주세요. 모르면 모른다고 답하세요. \n\n 기반지식:"
for doc in resutls:
    system_input += '\n' + doc.page_content

system_input

In [ ]:
chain.invoke({"system_input": system_input, "user_input":"등장인물 중 디멘투스는 어떤 인물인가요?"})

In [ ]:
resutls = retriever.get_relevant_documents("퓨리오사는 어머니는 어떻게 되나요?")
system_input = "당신은 정직한 에이전트 입니다. 반드시 주어진 기반지식에 근거해서 최대한 짧은 문장으로 답해주세요. 모르면 모른다고 답하세요. \n\n 기반지식:"
for doc in resutls:
    system_input += '\n' + doc.page_content

system_input

In [ ]:
chain.invoke({"system_input": system_input, "user_input":"퓨리오사는 어머니는 어떻게 되나요?"})

RetrievalQA로 chain을 만든 후 사용자 질문만 입력하고 실행한다.

In [ ]:
qa = RetrievalQA.from_chain_type(llm=watsonx_llama2_korean, chain_type="stuff", retriever=retriever)

In [ ]:
qa.invoke("퓨리오사가 한 쪽 팔을 잃게되는 이유는 뭔가요?")

In [ ]:
qa.invoke("등장인물 중 디멘투스는 어떤 역할인가요?")

In [ ]:
qa.invoke("퓨리오사의 어머니는 어떻게 되나요?")